# Project TT Instagram Database
- This notebook shows how to create a database from a bunch of instagram media posts

In [ ]:
# don't worry about these
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys
PROJECT_TT_ROOT = os.path.abspath('../..')  # project root
sys.path.append('..')  # add backend to python path

import pathlib
from time import sleep

from igramscraper.instagram import Instagram
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd  # pandas is a library that helps us manage spreadsheet-like data efficiently
from PIL import Image
%matplotlib inline  

# import previously defined functions
from core.utils import get_thumbnail, show_thumbnail, imresize
from core.instagram import get_media_by_url
from core.envs import IMAGE_DIR, THUMBNAIL_DIR

In [ ]:
instagram = Instagram(sleep_between_requests=15)

While our database is small, we will store our data as [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) file.

Let's create a dataset with 50 instagram posts!

In [ ]:
medias = instagram.get_medias_by_tag('london', count=50)  # this will take some time!

Now that they are scraped, lets save them in a database.
- We will first create pandas.Dataframe and put it into table-like format
- We will then use pandas functionality `to_csv` to generate the csv file representing our database

Recall the attributes we wanted to save from "[2] Instagram Data Attributes.ipynb". Note that currently there is no "location" information -- we will worry about this later.

In [ ]:
columns = [
    'media_id', 
    'media_code', 
    'media_link', 
    'user_id',
    'username',
    'user_full_name',
    'type', 
    'created_time',
    'likes_count', 
    'img_thumbnail_url', 
    'img_highres_url', 
    'carousel_ids',
    'carousel_types',
    'carousel_thumbnail_urls', 
    'carousel_highres_urls', 
    'caption',
    'comments_count',
    'comments',
    'location_id',
    'location_name',
    'location_slug',
]

Now we will write a function which will convert the "Media" object (that instagram scraper API is using to represent data) to a list of attributes that we are interested in, specified by `columns` above.

In [ ]:
def media_to_row(media):
    """ Convert media object read from instagram scrape API 
    to a row in dataset, with default values.
    """
    if hasattr(media, 'thumbnail_src'):
        media.image_thumbnail_url = media.thumbnail_src
        
    # check if media has carousels extracted
    if not hasattr(media, 'carousel_ids'):
        media.carousel_ids = ''
        media.carousel_types = ''
        media.carousel_thumbnail_urls = ''
        media.carousel_image_highres_urls = ''

    row = [
        media.identifier,
        media.short_code,
        media.link,
        media.owner.identifier,
        media.owner.username,
        media.owner.full_name,
        media.type,
        media.created_time,
        media.likes_count,
        media.image_thumbnail_url,
        media.image_high_resolution_url,
        media.carousel_ids,
        media.carousel_types,
        media.carousel_thumbnail_urls,
        media.carousel_image_highres_urls,
        media.caption,
        media.comments_count,
        media.comments,
        media.location_id,  
        media.location_name,
        media.location_slug,
    ]
    return row

Lets try to convert one and see how this looks like

In [ ]:
m = medias[0]

In [ ]:
sample_row = media_to_row(m)
for k, v in zip(columns, sample_row):  # zip allows us to iterate over "columns" and "row" simultaneously
    print(f'[{k}]: {v}')

Now let's convert the rest of the data and create `pandas.Dataframe` (a spreadsheet).

In [ ]:
rawdata = []
for m in medias:
    rawdata.append(media_to_row(m))

In [ ]:
df = pd.DataFrame(rawdata, columns=columns)
df.head(3)  # sneak peek first three items

In [ ]:
# save the dataframe as csv
data_path = pathlib.Path('tmp/data')
data_path.mkdir(parents=True, exist_ok=True)
df.to_csv(data_path / 'london.csv', quotechar="'")

Done! 

For fun... lets check what's the most liked photo in this 50 posts

In [ ]:
df_sorted = df.sort_values(by=['likes_count'], ascending=False).head(3)
for i in range(3):
    curr_data = df_sorted.iloc[i]
    url = curr_data.img_thumbnail_url
    best_thumbnail = get_thumbnail(url)
    show_thumbnail(best_thumbnail, f'#{i+1}: {curr_data.likes_count} likes')

Was it what you expected?

[Note]
When our database grows, CSV will no longer be efficient for accessing the data. Eventually we want a proper database. Options are:
- SQL,
- NoSQL,
- GraphQL, 
- etc..,

But we don't want to worry about that now for MVP